In [16]:
import sklearn
import pandas as pd
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression


## 1. 生成数据

#### 
n_samples：样本数<br>
n_features：特征数<br>
n_informative：有效特征个数<br>
n_redundant：冗余特征个数（有效特征的随机组合）<br>
n_classes：样本类别<br>
random_state：随机状态，用于确定创建数据集的随机数的生成<br>

In [17]:
# 生成数据
from collections import Counter
from sklearn.datasets import make_classification   
# 定义数据集
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)

# 总结数据集
print(X.shape, y.shape)
print(Counter(y))


(1000, 10) (1000,)
Counter({1: 334, 2: 334, 0: 332})


#### 上面的结果表示，属于1类的样本个数为334，属于2类的样本数为334，属于3类的样本数为332
这个数据集有1000行和10列（自变量的个数）<br>
Logit模型是通过LogisticsRegression 类在scikit-learn库中支持的<br>


## 2. 配置Logit模型
multi_class='multinomial'  ——参数设置为多项式<br>
solver='lbfgs'——设置为支持多项式logit模型的求解器<br>
上面就是在配置多项逻辑模型<br>

In [19]:
# 定义多项式 Logit模型模型
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
#该多项式 Logit模型模型将适合使用交叉熵损失，并将预测每个整数编码类标签的整数值。

## 3. 下面看一下如何在生成的多分类数据集上计算多项logit模型

#### 使用k-ford交叉验证 来评价分类模型是很好的一个做法；
分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。<br>
将使用10个folds进行三次重复，这是一个很好的默认值，并使用分类精度来评估模型的性能<br>
下面是评价多项式分类Logit模型的完整例子

In [20]:
# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')  
# define the model evaluation procedure （定义模型评估程序） n_splits 就是K-flods中的K值；n_repeats是交叉验证的次数
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores （评估模型并收集分数）
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance 
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))  

Mean Accuracy: 0.681 (0.042)


#### 运行上面的例子，可以显示评估过程的平均分类精度
从上面可以看出，有68.1%的平均分类准确度<br>
我们可以选择上面的模型作为我们的最终模型，然后对新数据进行预测；<br>

这可以对所有可用数据进行模型拟合，然后调用pridict()函数对新数据进行预测;<br>

## 4. 用多项logit模型对新数据进行预测

In [23]:
# 对整个数据集进行拟合（fit）
model.fit(X, y)

#定义一行新的数据
#注意要有10个，因为我们有10个自变量
row = [1.89149379, -0.39847585, 1.63856893, 0.01647165, 1.51892395, -3.52651223, 1.80998823, 0.58810926, -0.02542177, -0.52835426]

#预测新数据的分布概率
yhat = model.predict_proba([row])

#输出预测结果
print('Predicted Probabilities: %s' % yhat[0])

Predicted Probabilities: [0.16470456 0.50297138 0.33232406]


#### 因为我们的每次选择的时候有3个类别，则上面的三个值分别对应的是，选择不同类别的概率；
可以看出 ，类1（索引从0开始）对应的概率最大，约为0.5；